In [1]:
import gymnasium as gym
import numpy as np 
import matplotlib.pyplot as plt
import random
import pandas as pd
# import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.optimizers.legacy import Adam
# from keras import __version__
# tf.keras.__version__ = __version__
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
import time
import seaborn as sns
from utils import argmax
from tqdm import tqdm
from scipy.interpolate import make_interp_spline

ImportError: cannot import name '__version__' from 'tensorflow.keras' (/Users/makotomiyazaki/.pyenv/versions/3.11.5/envs/rl/lib/python3.11/site-packages/keras/api/_v2/keras/__init__.py)

In [ ]:
# 1. Load Environment and Q-table structure
env = gym.make('ALE/SpaceInvaders-v5')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [3]:
print(height, width, channels, actions)

210 160 3 6


In [4]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [5]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides = (4,4), activation = 'relu', input_shape = (3, height, width, channels))) # output = 51 * 39
    model.add(Convolution2D(64, (4,4), strides = (2,2), activation = 'relu')) # output = 24 * 18
    model.add(Convolution2D(64, (3,3), activation = 'relu')) # output = 22 * 16
    model.add(Flatten()) # output = 22 * 16 * 64 = 22528
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(216, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [6]:
model = build_model(height, width, channels, actions)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten (Flatten)           (None, 67584)             0         
                                                                 
 dense (Dense)               (None, 512)               34603520  
                                                                 
 dense_1 (Dense)             (None, 216)               110808    
                                                                 
 dense_2 (Dense)             (None, 6)                 1

In [8]:
def build_agent(model, actions):
    # LinearAnnealedPolicy: decrease epsilon linearly from 1 to .1
    policy = LinearAnnealedPolicy( 
        EpsGreedyQPolicy(), 
        attr='eps', 
        value_max=1., 
        value_min=.1, 
        value_test=.2, 
        nb_steps=10000
        )
    
    memory = SequentialMemory(
        limit=1000000,  # This memory will store up to 1 million of the most recent experiences.
        window_length=3 # how many of the most recent observations will be used as the input for the neural network.
        )
    
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy, 
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions, 
        nb_steps_warmup=1000,
        )
    return dqn


In [9]:
dqn = build_agent(model, actions)
dqn.compile(optimizer=Adam(learning_rate=1e-4))

ValueError: Requested the deserialization of a Lambda layer with a Python `lambda` inside it. This carries a potential risk of arbitrary code execution and thus it is disallowed by default. If you trust the source of the saved model, you can pass `safe_mode=False` to the loading function in order to allow Lambda layer loading.